# Read Files How To

I pulled down a single day's files into an S3 bucket
* aws s3 cp s3://gdelt-open-data/v2/events/ s3://labadie.gdelt/ --recursive --exclude "*" --include "20180619??????.export.csv"
* I don't think you can use wildcards with hadoop distcp

I then brought those into hdfs
  * hadoop distcp s3://labadie.gdelt/ gdelt/

# Get GDELT Events Schema

In [68]:
import urllib.request  # lib that handles URLs
#import requests
import io

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data))
formats["dataType"].replace({"INTEGER":"IntegerType",
                             "FLOAT":"LongType",
                             "STRING":"StringType"},inplace=True)

### Try to build the schema

In [101]:
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

gdeltschema = []
for row in formats.iterrows():
    sf = StructField(row[1]["tableId"],
                     StringType(),
                     #globals()[row[1]["dataType"]](),
                     True)
    gdeltschema.append(sf)
    
gdeltschema=StructType(gdeltschema)

# Spark Stuff

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [108]:
df = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .schema(gdeltschema) \
    .load("gdelt/*.csv")
df.cache()

DataFrame[GLOBALEVENTID: string, SQLDATE: string, MonthYear: string, Year: string, FractionDate: string, Actor1Code: string, Actor1Name: string, Actor1CountryCode: string, Actor1KnownGroupCode: string, Actor1EthnicCode: string, Actor1Religion1Code: string, Actor1Religion2Code: string, Actor1Type1Code: string, Actor1Type2Code: string, Actor1Type3Code: string, Actor2Code: string, Actor2Name: string, Actor2CountryCode: string, Actor2KnownGroupCode: string, Actor2EthnicCode: string, Actor2Religion1Code: string, Actor2Religion2Code: string, Actor2Type1Code: string, Actor2Type2Code: string, Actor2Type3Code: string, IsRootEvent: string, EventCode: string, EventBaseCode: string, EventRootCode: string, QuadClass: string, GoldsteinScale: string, NumMentions: string, NumSources: string, NumArticles: string, AvgTone: string, Actor1Geo_Type: string, Actor1Geo_FullName: string, Actor1Geo_CountryCode: string, Actor1Geo_ADM1Code: string, Actor1Geo_ADM2Code: string, Actor1Geo_Lat: string, Actor1Geo_Lon

In [110]:
df.count()

192540

In [111]:
df.show(5)

+-------------+--------+---------+----+------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+-----------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+
|GLOBALEVENTID| 